In [1]:
import subprocess as sp
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns  # for aesthetic

import neuraltda.topology2 as tp2
import neuraltda.spectralAnalysis as sa
import neuraltda.simpComp as sc
import numpy as np
import tempfile
import h5py as h5
import glob
import os
%matplotlib inline

sns.set_style('ticks')

/home/brad/.conda/envs/neuraltda-3/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
def rejection_sampling(command, seed=0):
    # Call sampler with subprocess
    proc = sp.run(command, stdout=sp.PIPE)
    # Read output as a facet list 
    facet_list = []
    for line in proc.stdout.decode().split("\n")[1:-1]:
        if line.find("#") == 0:
            yield facet_list
            facet_list = []
        else:
            facet_list.append([int(x) for x in line.strip().split()])
    yield facet_list
    
def prepare_scm_initial_condition(binmat, **kwargs):
    
    facets = sc.binarytomaxsimplex(binmat, rDup=True, **kwargs)
    with tempfile.NamedTemporaryFile(mode='w+t', delete=False) as f: 
        fname = f.name
        for facet in facets:
            fstr = str(facet)
            fstr = fstr.replace('(', '')
            fstr = fstr.replace(')', '')
            fstr = fstr.replace(',', '')
            f.write(fstr+'\n')
    return fname

def prepare_scm_command(facet_file, nsamps):
    
    command = ['/home/brad/bin/mcmc_sampler', facet_file, '-t', str(nsamps)]
    return command

def extract_population_tensor(binned_data_file, stim, shuffle=False, clusters=None):
    print('Extracting Population Activity Tensor...')
    with h5.File(binned_data_file, 'r') as bdf:
        binned_clusters = np.array(bdf[stim]['clusters'])
        poptens = np.array(bdf[stim]['pop_tens'])
        print('Stim: {}, Clusters:{}'.format(stim, str(clusters)))
        try:
            if clusters is not None:
                poptens = poptens[np.in1d(binned_clusters, clusters), :, :]
                print("Selecting Clusters: poptens:" + str(np.shape(poptens)))
            (ncell, nwin, ntrial) = np.shape(poptens)
        except (ValueError, IndexError):
            print('Population Tensor Error')
            return []
        if shuffle:
            poptens = tp2.build_shuffled_data_tensor(poptens, 1)
            poptens = poptens[:, :, :, 0]
        if  nwin == 0:
            return []
    return poptens
            
def compute_trial_final_bettis(poptens, thresh, trial):
    popmat = poptens[:, :, trial]
    nclus = num_cells(poptens)
    popmat_binary = sc.binnedtobinary(popmat, thresh)
    bettis = tp2.calc_bettis(popmat, np.arange(nclus), '/home/brad/betti_pfile.txt', thresh)
    final_bettis = bettis[-1][1]
    return final_bettis

def num_trials(poptens):
    (ncells, nwin, ntrials) = np.shape(poptens)
    return ntrials

def num_win(poptens):
    (ncells, nwin, ntrials) = np.shape(poptens)
    return nwin

def num_cells(poptens):
    (ncells, nwin, ntrials) = np.shape(poptens)
    return ncells
    
    
def calc_scm_betti_distribution(poptens, thresh, trial, nsamples):
    popmat = poptens[:, :, trial]
    popmat_bin = sc.binnedtobinary(popmat, thresh)
    fname = prepare_scm_initial_condition(popmat_bin)
    cmd = prepare_scm_command(fname, nsamples)
    samples = rejection_sampling(cmd)
    sample_bettis = []
    for sample in samples:
        bettis=[]
        cgs = [[1, x] for x in sample]
        tp2.build_perseus_input(cgs, '/home/brad/betti_pfile.txt')
        betti_file = tp2.run_perseus('/home/brad/betti_pfile.txt')
        try:
            with open(betti_file, 'r') as bf:
                for bf_line in bf:
                    if len(bf_line) < 2:
                        continue
                    betti_data = bf_line.split()
                    filtration_time = int(betti_data[0])
                    betti_numbers = map(int, betti_data[1:])
                    bettis.append([filtration_time, betti_numbers])
        except:
            bettis.append([-1, [-1]])
        sample_bettis.append(bettis)
    return np.array(sample_bettis)

In [3]:
from importlib import reload
reload(tp2.events)
reload(tp2.core)
reload(tp2)

<module 'neuraltda.topology2' from '/home/brad/code/NeuralTDA/neuraltda/topology2.py'>

In [4]:
winsize = 10.0
segment_info = [0, 0]
pover = 0.5
block_path = '/home/brad/krista/B1075/P01S03/'
# Bin and compute SCG
bfdict = tp2.dag_bin(block_path, winsize, segment_info, cluster_group=['Good', 'MUA'], dt_overlap=pover*winsize, comment='fulltrial')
binned_data_file = glob.glob(os.path.join(bfdict['raw'], '*.binned'))[0]


--- Logging error ---
Traceback (most recent call last):
  File "/home/brad/.conda/envs/neuraltda-3/lib/python3.6/logging/__init__.py", line 994, in emit
    stream.write(msg)
AttributeError: 'int' object has no attribute 'write'
Call stack:
  File "/home/brad/.conda/envs/neuraltda-3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/brad/.conda/envs/neuraltda-3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/brad/.conda/envs/neuraltda-3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/brad/.conda/envs/neuraltda-3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/brad/.conda/envs/neuraltda-3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/brad/.conda/envs/neuraltda-3/lib/python3.6/site-pac

In [5]:
thresh = 13
nsamples = 10

with h5.File(binned_data_file, 'r') as bdf:
    stims = bdf.keys()
    for stim in stims:
        print(stim)
        stim_poptens = extract_population_tensor(binned_data_file, stim)
        stim_poptens_shuffled = extract_population_tensor(binned_data_file, stim, shuffle=True)
        ntrials = num_trials(stim_poptens)
        for trial in range(ntrials):
            
            stim_bettis = compute_trial_final_bettis(stim_poptens, thresh, trial)
            
            stim_shuffled_bettis = compute_trial_final_bettis(stim_poptens_shuffled, thresh, trial)
            
            scm_bettis = calc_scm_betti_distribution(stim_poptens, thresh, trial, nsamples)
        
        

A_40k
Extracting Population Activity Tensor...
Stim: A_40k, Clusters:None
Extracting Population Activity Tensor...
Stim: A_40k, Clusters:None
B_40k
Extracting Population Activity Tensor...
Stim: B_40k, Clusters:None
Extracting Population Activity Tensor...
Stim: B_40k, Clusters:None
C_40k
Extracting Population Activity Tensor...
Stim: C_40k, Clusters:None
Extracting Population Activity Tensor...
Stim: C_40k, Clusters:None
D_40k
Extracting Population Activity Tensor...
Stim: D_40k, Clusters:None
Extracting Population Activity Tensor...
Stim: D_40k, Clusters:None
E_40k
Extracting Population Activity Tensor...
Stim: E_40k, Clusters:None
Extracting Population Activity Tensor...
Stim: E_40k, Clusters:None
F_40k
Extracting Population Activity Tensor...
Stim: F_40k, Clusters:None
Extracting Population Activity Tensor...
Stim: F_40k, Clusters:None
G_40k
Extracting Population Activity Tensor...
Stim: G_40k, Clusters:None
Extracting Population Activity Tensor...
Stim: G_40k, Clusters:None
H_40k


In [9]:
list(stim_bettis)

[1]

In [8]:
scm_bettis

array([ list([[1, <map object at 0x7f58658a75c0>], [2, <map object at 0x7f58658a7358>], [3, <map object at 0x7f58658a7198>], [4, <map object at 0x7f58658a7f98>], [5, <map object at 0x7f58658a7f28>], [6, <map object at 0x7f58658a7e80>], [7, <map object at 0x7f58658a76a0>], [8, <map object at 0x7f58658a75f8>], [10, <map object at 0x7f58658a7860>]]),
       list([[1, <map object at 0x7f58658a72b0>], [2, <map object at 0x7f58658a7eb8>], [3, <map object at 0x7f58658a7278>], [4, <map object at 0x7f58658a7a20>], [5, <map object at 0x7f58658a7da0>], [6, <map object at 0x7f58658a7ef0>], [7, <map object at 0x7f58658a7400>], [8, <map object at 0x7f58658ac1d0>], [9, <map object at 0x7f58658acc88>]]),
       list([[1, <map object at 0x7f58658a7080>], [2, <map object at 0x7f58658a7fd0>], [3, <map object at 0x7f586593ea58>], [4, <map object at 0x7f586593ed68>], [5, <map object at 0x7f586593ee10>], [6, <map object at 0x7f586593e828>], [7, <map object at 0x7f586593e2e8>], [8, <map object at 0x7f586593e